## Create NARR day-of year climos
##### Purpose of this notebook is to create a jul day climatology (365, lat, lon) of any narr_conv variable
##### Dr. Victor Gensini (Winter 2017)
##### Department of Geographic and Atmospheric Sciences
<img style="float:left;height:40px"src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQthtdrc2lZyEOC1iiwubFyvNfzJE7q7_GCTdlFF3peKckmi7cN">

In [1]:
import datetime, os, calendar
import numpy as np
import pandas as pd
from netCDF4 import date2num,date2index
from netCDF4 import Dataset
from IPython.display import Image

### What variable would you like to make climatology for?

In [2]:
var_name = 'dewp2m'
minexp=0
maxexp=500

### Read in one file to capture lat/lons

In [3]:
nc1 = Dataset('narr_latlon.nc','r',format='NETCDF4_CLASSIC')
lats = nc1.variables["lats"][0][:][:]
lons = nc1.variables["lons"][0][:][:]
nc1.close()

In [4]:
scp_climo = np.empty((0,277,349))
scp_final = np.empty((0,38,277,349))
nc = Dataset('/home/data/bwd500hgtdewp/controls_1979_2016.nc','r',format='NETCDF4_CLASSIC')

In [5]:
###############################
#Enter the start and end date:#
######################################
#start_input = '1979110100' #YYYYMMDDHH
#end_input  =  '2016110100' #YYYYMMDDHH
######################################
months=['01','02','03','04','05','06','07','08','09','10','11','12']
for month in months:
    if month=='01' or month=='03' or month=='05' or month=='07' or month=='08' or month=='10' or month=='12':
        emon = 31
    elif month=='04' or month=='06' or month=='09' or month=='11':
        emon = 30
    if month=='02':
        emon = 28
    daycount=1
    while daycount <= emon:
        start_input = '1979'+month+'%02d'%daycount+'21'    #YYYYMMDDHH
        end_input  =  '2016'+month+'%02d'%daycount+'21' #YYYYMMDDHH
        begdate = datetime.datetime.strptime(start_input,"%Y%m%d%H")
        enddate = datetime.datetime.strptime(end_input,"%Y%m%d%H")
        dates = []
        while begdate <= enddate:
            if calendar.isleap(begdate.year) == True:   
                dates.append(begdate)
                begdate+=datetime.timedelta(days=1)
            else:
                dates.append(begdate)
            begdate+=datetime.timedelta(days=365)
        #do things here
        #print dates
        #print len(dates)
        for i,dt in enumerate(dates):
            idex = date2index(dt,nc['time'])
            scp = nc.variables[var_name][idex][:][:]
            #cin  = nc.variables["sbcin"][idex][:][:]
            #cin mask
            #term5 = np.fabs(cin)
            #term5[np.fabs(cin)>50]=0.
            #term5[np.fabs(cin)<=50]=1.
            #scp = scp * term5
            scp[scp<minexp]=0.
            scp[scp>=maxexp]=0.
            scp_climo = np.append(scp_climo,[scp],axis=0)
            #print scp_climo.shape
        scp_final=np.append(scp_final,[scp_climo],axis=0)
        scp_climo = np.empty((0,277,349))
        daycount+=1
print (scp_final.shape)

/home/vgensini/.conda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:41: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.


(365, 38, 277, 349)


In [6]:
#nc.close()
mean_scp_daily = np.mean(scp_final,axis=1)
#stdev_scp_daily = np.std(scp_final,axis=1)
#log_values = np.log(scp_final)
#mean_scp_daily_log = np.mean(log_values,axis=1)
#stdev_scp_daily_log = np.std(log_values,axis=1)
print (mean_scp_daily.shape)
#print stdev_scp_daily.shape
mean_scp_daily.dump('narr_doy_'+var_name+'_1979_2016.npy') #Daily Mean
#stdev_scp_daily.dump('narr_stdev_scp_daily.npy') #Daily STDEV
#mean_scp_daily_log.dump('narr_mean_scp_daily_log.npy') 
#stdev_scp_daily_log.dump('narr_stdev_scp_daily_log.npy') 

(365, 277, 349)


In [7]:
print (mean_scp_daily[0,:,:])

[[ 295.72574013  295.71710526  295.71710526 ...,    0.            0.            0.        ]
 [ 295.65830592  295.65830592  295.64679276 ...,    0.            0.            0.        ]
 [ 295.62314967  295.61821546  295.61821546 ...,    0.            0.            0.        ]
 ..., 
 [   0.            0.            0.         ...,    0.            0.            0.        ]
 [   0.            0.            0.         ...,    0.            0.            0.        ]
 [   0.            0.            0.         ...,    0.            0.            0.        ]]
